# Анализ категорий пользователей

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
#path_to_db_local = 'telecomm_csi.db'
path_to_db_local = r'C:\Users\Владислав\Documents\dat_an\telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT 
u.user_id,
u.lt_day,
u.age,
u.gender_segment,
u.os_name,
u.cpe_type_name,
u.nps_score,
lc.country,
lc.city,
substr(ags.title,3) AS agesegment,
substr(ts.title,3) AS trafficsegment,
substr(ls.title,3) AS lifetimesegment,
CASE
    WHEN u.lt_day<=365 THEN "новый"
    ELSE "старый"
END AS is_new,
CASE
    WHEN u.nps_score IN (9,10) THEN "сторонники"
    WHEN u.nps_score IN (7,8) THEN "нейтралы"
    ELSE "критики"
END AS nps_group,
CASE
    WHEN u.gender_segment=1 THEN "жен"
    WHEN u.gender_segment=0 THEN "муж"
    ELSE "не определен"
END AS gender
FROM user AS u
LEFT JOIN location AS lc ON u.location_id=lc.location_id
LEFT JOIN age_segment AS ags ON u.age_gr_id=ags.age_gr_id 
LEFT JOIN traffic_segment AS ts ON u.tr_gr_id=ts.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,nps_score,country,city,agesegment,trafficsegment,lifetimesegment,is_new,nps_group,gender
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,10,Россия,Уфа,45-54,1-5,36+,старый,сторонники,жен
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,10,Россия,Киров,45-54,1-5,36+,старый,сторонники,муж
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,10,Россия,Москва,55-64,20-25,13-24,старый,сторонники,муж
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,10,Россия,РостовнаДону,35-44,0.1-1,36+,старый,сторонники,жен
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,10,Россия,Рязань,16-24,5-10,36+,старый,сторонники,муж
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,6,Россия,Омск,35-44,5-10,36+,старый,критики,жен
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,10,Россия,Уфа,45-54,10-15,36+,старый,сторонники,муж
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,10,Россия,Москва,35-44,45-50,7-12,новый,сторонники,муж
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,10,Россия,СанктПетербург,35-44,1-5,36+,старый,сторонники,жен
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,9,Россия,Волгоград,45-54,15-20,36+,старый,сторонники,жен


In [9]:
df.head()

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,nps_score,country,city,agesegment,trafficsegment,lifetimesegment,is_new,nps_group,gender
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,10,Россия,Уфа,45-54,1-5,36+,старый,сторонники,жен
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,10,Россия,Киров,45-54,1-5,36+,старый,сторонники,муж
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,10,Россия,Москва,55-64,20-25,13-24,старый,сторонники,муж
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,10,Россия,РостовнаДону,35-44,0.1-1,36+,старый,сторонники,жен
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,10,Россия,Рязань,16-24,5-10,36+,старый,сторонники,муж


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_id          502493 non-null  object 
 1   lt_day           502493 non-null  int64  
 2   age              501939 non-null  float64
 3   gender_segment   501192 non-null  float64
 4   os_name          502493 non-null  object 
 5   cpe_type_name    502493 non-null  object 
 6   nps_score        502493 non-null  int64  
 7   country          502493 non-null  object 
 8   city             502493 non-null  object 
 9   agesegment       502493 non-null  object 
 10  trafficsegment   502493 non-null  object 
 11  lifetimesegment  502493 non-null  object 
 12  is_new           502493 non-null  object 
 13  nps_group        502493 non-null  object 
 14  gender           502493 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 57.5+ MB


In [12]:
df=df.dropna()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501152 entries, 0 to 502492
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_id          501152 non-null  object 
 1   lt_day           501152 non-null  int64  
 2   age              501152 non-null  float64
 3   gender_segment   501152 non-null  float64
 4   os_name          501152 non-null  object 
 5   cpe_type_name    501152 non-null  object 
 6   nps_score        501152 non-null  int64  
 7   country          501152 non-null  object 
 8   city             501152 non-null  object 
 9   agesegment       501152 non-null  object 
 10  trafficsegment   501152 non-null  object 
 11  lifetimesegment  501152 non-null  object 
 12  is_new           501152 non-null  object 
 13  nps_group        501152 non-null  object 
 14  gender           501152 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 61.2+ MB


In [14]:


df.to_csv('telecomm_csi_tableau3.csv', index=False)

<div class="alert alert-info"  style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
    
<h2> <a class="tocSkip"> </h2>

Ссылки

Ссылка на дашборды

https://public.tableau.com/views/finaltableauproject_17068788184140/sheet13?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

Ссылка на презентацию с ответами на вопросы

https://cloud.mail.ru/public/3aVW/9UPwoDkYZ

<div class="alert alert-info"  
